In [1]:
pip install -q "qdrant-client[fastembed]>=1.14.2"

You should consider upgrading via the '/Volumes/Sozhan-External/Repository/llm-zoomcamp-2025/llmenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np
import requests

/Volumes/Sozhan-External/Repository/llm-zoomcamp-2025/llmenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
query='I just discovered the course. Can I join now?'
model_name = 'jinaai/jina-embeddings-v2-small-en'

model = TextEmbedding(model_name=model_name)
embeddings_query = list(model.embed([query]))
len(embeddings_query[0])

512

In [6]:
round(np.min(embeddings_query[0]), 3)

np.float64(-0.117)

In [11]:
np.linalg.norm(embeddings_query[0])

np.float64(0.9999999999999999)

In [12]:
embeddings_query[0].dot(embeddings_query[0])

np.float64(0.9999999999999998)

In [13]:
doc = 'Can I still join the course after the start date?'

In [14]:
embeddings_doc = list(model.embed([doc]))

In [15]:
embeddings_doc[0].dot(embeddings_query[0])

np.float64(0.9008529113860175)

In [16]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]


In [17]:
embeddings_vector = []

for document in documents:
    embeddings_doc = list(model.embed([document['text']]))
    embeddings_vector.append(embeddings_doc[0])

cos_sim = np.array(embeddings_vector).dot(embeddings_query[0])
np.argmin(cos_sim)

np.int64(3)

In [18]:
embeddings_vector = []

for document in documents:
    full_text = document['question'] + ' ' + document['text']
    embeddings_doc = list(model.embed([full_text]))
    embeddings_vector.append(embeddings_doc[0])

cos_sim = np.array(embeddings_vector).dot(embeddings_query[0])
np.argmax(cos_sim)

np.int64(0)

In [19]:
models_list = TextEmbedding.list_supported_models()

dimensions = []
for model in models_list:
    dimensions.append(model['dim'])
    
models_list[np.array(dimensions).argmin()]

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

In [20]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
documents[:5]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [22]:
client = QdrantClient("http://localhost:6333")

EMBEDDING_DIMENSIONALITY = 384
collection_name = "ml-zoomcamp-rag"
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

model_name = 'BAAI/bge-small-en'
points = []
id_ = 0
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=text, model=model_name),
        payload={
            "text": text,
            "section": doc['section'],
            "course": doc['course']
        }
    )
    points.append(point)

client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_name
        ),
        limit=limit,
        with_payload=True
    )

    return results

result = search(query).points[0]

query

'I just discovered the course. Can I join now?'

In [25]:
result.payload["text"]

'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

In [ ]:
result.score